# 🔮 Demo Inference: Dự Đoán Doanh Thu Phim

Notebook này cho phép bạn nhập thông tin một bộ phim giả định (hoặc phim mới sắp ra mắt) và sử dụng các mô hình đã train để dự đoán doanh thu.

## 1. Setup & Load Model

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import joblib
from pathlib import Path

# Setup đường dẫn project
current_dir = Path(os.getcwd())
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
else:
    project_root = current_dir

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import src modules
from src.preprocessing import DataPreprocessor

print(f"Project Root: {project_root}")

In [ ]:
# Đường dẫn tới models
models_dir = project_root / "models"

# Load Preprocessor đã fit từ trước
preprocessor_path = models_dir / "preprocessor.pkl"
try:
    preprocessor = joblib.load(preprocessor_path)
    print("✅ Đã load Preprocessor thành công!")
except FileNotFoundError:
    print("❌ Không tìm thấy preprocessor.pkl. Bạn cần chạy pipeline training trước!")

# Load Model (Ví dụ: XGBoost - thường cho kết quả tốt nhất)
model_path = models_dir / "xgboost.pkl"
try:
    model = joblib.load(model_path)
    print(f"✅ Đã load Model XGBoost thành công!")
except FileNotFoundError:
    print("❌ Không tìm thấy model. Bạn cần chạy pipeline training trước!")

## 2. Nhập Thông Tin Phim

In [ ]:
# ==========================================
# 👇 NHẬP THÔNG TIN PHIM CỦA BẠN TẠI ĐÂY 👇
# ==========================================

movie_input = {
    'title': "Dune: Part Two",
    'budget': 190000000,          # 190 triệu USD
    'runtime': 166,               # Phút
    'release_date': "2024-03-01", # YYYY-MM-DD
    'vote_average': 8.5,          # Giả định điểm đánh giá
    'vote_count': 5000,           # Giả định số lượt vote
    'popularity': 500.0,          # Độ phổ biến
    'genres': "Science Fiction|Adventure",  # Các thể loại cách nhau bởi dấu |
    'overview': "Paul Atreides unites with Chani and the Fremen while on a warpath of revenge against the conspirators who destroyed his family." # Tóm tắt
}

# Chuyển thành DataFrame
df_input = pd.DataFrame([movie_input])
df_input

## 3. Dự Báo (Predict)

In [ ]:
# Preprocess dữ liệu đầu vào
# Lưu ý: Chúng ta dùng transform() chứ không fit()
try:
    X_new, _ = preprocessor.transform(df_input)
    print("Input Shape:", X_new.shape)

    # Predict (Kết quả trả về là log_revenue do lúc train chúng ta đã log transform y)
    log_prediction = model.predict(X_new)
    
    # Inverse Log Transform để ra số tiền thật
    revenue_prediction = np.expm1(log_prediction)[0]
    
    print("-"*40)
    print(f"🎬 Phim: {movie_input['title']}")
    print(f"💰 Dự đoán Doanh Thu: ${revenue_prediction:,.2f}")
    print(f"💸 Kinh Phí:         ${movie_input['budget']:,.2f}")
    
    roi = (revenue_prediction - movie_input['budget']) / movie_input['budget'] * 100
    print(f"📈 ROI Dự kiến:       {roi:.2f}%")
    print("-"*40)

except Exception as e:
    print(f"Lỗi khi dự đoán: {e}")